<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code uses Hidden Markov Models (HMM) to identify regimes in financial time series data. It downloads historical price data, calculates returns and ranges, and uses them as features for the HMM. The model is fitted with the features to identify different market states. The identified states are then plotted to visualize market regime changes. This is useful for understanding and predicting market behavior.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from hmmlearn import hmm

Download historical price data for SPY from Yahoo Finance

In [ ]:
data = yf.download("SPY")

Calculate log returns of the closing prices

In [ ]:
returns = np.log(data.Close / data.Close.shift(1))

Calculate the range as the difference between high and low prices

In [ ]:
range = (data.High - data.Low)

Concatenate returns and range into a single DataFrame and drop any missing values

In [ ]:
features = pd.concat([returns, range], axis=1).dropna()
features.columns = ["returns", "range"]

Initialize a Gaussian Hidden Markov Model with 3 states and fit it to the features

In [ ]:
model = hmm.GaussianHMM(
    n_components=3,
    covariance_type="full",
    n_iter=1000,
)
model.fit(features)

Predict the hidden states for the given features and store them in a Series

In [ ]:
states = pd.Series(model.predict(features), index=data.index[1:])
states.name = "state"

Plot a histogram of the hidden states

In [ ]:
states.hist()

Define a color map for the different states

In [ ]:
color_map = {
    0.0: "green",
    1.0: "orange",
    2.0: "red"
}

Concatenate the closing prices and the states, drop missing values, 
set state as a hierarchical index, unstack the state index, and plot the closing prices with different colors for each state

In [ ]:
(
    pd.concat([data.Close, states], axis=1)
    .dropna()
    .set_index("state", append=True)
    .Close
    .unstack("state")
    .plot(color=color_map)
)

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.